In [ ]:
%%time
import os
import nltk
import gzip
import json
import gensim
import pickle 
import pyLDAvis
import pyLDAvis.gensim
from pprint import pprint
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from tokenizers.pre_tokenizers import Sequence ,Whitespace, Punctuation


pyLDAvis.enable_notebook()


### Downloads ###
# nltk.download('brown')
# !pip install pyLDAvis
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Gift_Cards_5.json.gz       'Gift_Cards_5.json.gz'
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Books_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Home_and_Kitchen_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Sports_and_Outdoors_5.json.gz
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz
# ### ######### ###

chunk_size=5 * 1024 * 1024 
 
# with gzip.open("Sports_and_Outdoors_5.json.gz", 'rb') as f:
#     with open("Sports_and_Outdoors_5.json", 'wb') as f_out:
#         while True:
#             chunk = f.read(chunk_size)
#             if not chunk:
#                 break
#             f_out.write(chunk)
# print("finished_sports")
# with gzip.open("Home_and_Kitchen_5.json.gz", 'rb') as f:
#     with open("Home_and_Kitchen_5.json", 'wb') as f_out:
#         while True:
#             chunk = f.read(chunk_size)
#             if not chunk:
#                 break
#             f_out.write(chunk)
# print("finished_Home")

# with gzip.open("Books_5.json.gz", 'rb') as f:
#     with open("Books_5.json", 'wb') as f_out:
#         while True:
#             chunk = f.read(chunk_size)
#             if not chunk:
#                 break
#             f_out.write(chunk)
# print("finished_books")

with gzip.open("Electronics_5.json.gz", 'rb') as f:
    with open("Electronics_5.json", 'wb') as f_out:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            f_out.write(chunk)
print("finished_eletric")
 

training_data_outdoors = []
training_data_kitchen=[]
# training_data_books = []
# training_data_electronics = []

     
with open('Home_and_Kitchen_5.json') as f:
    for l in f:
        training_data_kitchen.append(json.loads(l.strip()))
print("added_home_lst")
with open('Sports_and_Outdoors_5.json') as f:
    for l in f:
        training_data_outdoors.append(json.loads(l.strip()))
print("added_sports_lst")      

# with open('Books_5.json') as f:
#     for l in f:
#         training_data_books.append(json.loads(l.strip()))
# print("added_books_lst")           
# with open('Electronics_5.json') as f:
#     for l in f:
#         training_data_electronics.append(json.loads(l.strip()))
        
complete_test=[]
complete_test.extend(training_data_outdoors[:int(0.8*len(training_data_outdoors))])
complete_test.extend(training_data_kitchen[:int(0.8*len(training_data_kitchen))])
#complete_test.extend(training_data_books[:int(0.8*len(training_data_books))])
# complete_test.extend(training_data_electronics[:int(0.8*len(training_data_electronics))])


simplified_data=[]
for i in range(len(complete_test)):
    simplified_data.append(complete_test[i].get("summary",'')+' '+complete_test[i].get("reviewText",''))
# print(len(simplified_data))


def sent_to_words(data):
    for obj in data:
        yield(gensim.utils.simple_preprocess(str(obj), deacc=True))  # deacc=True removes punctuations


def stopwordscleaner(data):
    stp = set(stopwords.words('english'))
    return [[tok for tok in obj if tok.lower() not in stp] for obj in data]
    
clean_words=sent_to_words(simplified_data)
clean_data=stopwordscleaner(clean_words)


id2word=corpora.Dictionary(clean_data)
reviews=clean_data
review_corpus=[id2word.doc2bow(review) for review in reviews]

topic=200
lda=gensim.models.ldamulticore.LdaMulticore(corpus=review_corpus,id2word=id2word,num_topics=topic,workers=4)
# pprint(lda.print_topics())

LDAvis_prepared = pyLDAvis.gensim.prepare(lda, review_corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared,'lda_model'+'.html')
LDAvis_prepared

--2023-11-20 19:31:08--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1251876861 (1.2G) [application/x-gzip]
Saving to: ‘Electronics_5.json.gz’

100%[====================================>] 1,251,876,861 28.5MB/s   in 40s    

2023-11-20 19:31:49 (29.6 MB/s) - ‘Electronics_5.json.gz’ saved [1251876861/1251876861]

finished_eletric
added_home_lst
added_sports_lst
